In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
%reset -f

In [ ]:
from collections import abc

import numpy as np
from scipy import fft

from harmonic_balance import arclength_continuation as alc
from harmonic_balance import continuation, freq, solve, utils

ndarray = np.ndarray

## Define nonlinear force

In [ ]:
def get_slice(dof: int, N: int):
    return np.s_[dof * N : (dof + 1) * N]


def get_nonlinear(
    factor: float,
) -> tuple[
    abc.Callable[[ndarray, ndarray, int], ndarray],
    abc.Callable[[ndarray, ndarray, int], ndarray],
    abc.Callable[[ndarray, ndarray, int], ndarray],
]:
    dof = 0
    s_ = lambda N: get_slice(dof, N)

    def f_nl(x: ndarray, xdot: ndarray, N: int) -> ndarray:
        res = np.zeros_like(x, dtype=float)
        res[s_(N)] += factor * x[s_(N)] ** 3
        return res

    def df_nl_dx(x: ndarray, xdot: ndarray, N: int) -> ndarray:
        res = np.zeros_like(x, dtype=float)
        res[s_(N)] += factor * 3 * x[s_(N)] ** 2
        return np.diag(res)

    def df_nl_d_xdot(x: ndarray, xdot: ndarray, N: int) -> ndarray:
        shape = x.shape[0]
        return np.zeros((shape, shape), dtype=float)

    return f_nl, df_nl_dx, df_nl_d_xdot

## Define system

In [ ]:
n = 2

m1, m2 = 1, 2
c1, c2, c3 = 1, 0.5, 2
k1, k2, k3 = 1, 0.5, 2

M = np.diag([m1, m2])
C = np.array([[c1 + c2, -c2], [-c2, c2 + c3]])
K = np.array([[k1 + k2, -k2], [-k2, k2 + k3]])

ks = (1, 1)
dofs = (0, 1)
is_cosines = (1, 0)
coefficients = (2, 1)

factor = 10
f_nl, df_nl_dx, df_nl_d_xdot = get_nonlinear(factor)

## Set harmonic balance parameters

In [ ]:
NH = 30
N = 128

b_ext = freq.get_b_ext(NH, n, ks, dofs, is_cosines, coefficients)

## Run harmonic balance

In [ ]:
s = 0.05

tol = 1e-11
max_iter = 100

omega0 = 2
A0 = freq.get_A(omega0, NH, M, C, K)
initial_guess = freq.solve_linear_system(A, b_ext)
z0, *_ = solve.solve_nonlinear(
    omega0,
    initial_guess,
    A0,
    b_ext,
    f_nl,
    df_nl_dx,
    df_nl_d_xdot,
    NH,
    n,
    N,
    tol,
    max_iter=max_iter,
)

omega1 = omega0 + s
A1 = freq.get_A(omega1, NH, M, C, K)
initial_guess = freq.solve_linear_system(A1, b_ext)
z1, *_ = solve.solve_nonlinear(
    omega1,
    initial_guess,
    A0,
    b_ext,
    f_nl,
    df_nl_dx,
    df_nl_d_xdot,
    NH,
    n,
    N,
    tol,
    max_iter=max_iter,
)

## Try continuation

In [ ]:
alc_tol = 1e-6
alc_max_iter = 20

s = 0.01

y1 = np.concat((z1, [omega1]))
y0 = np.concat((z0, [omega0]))
y_pred = alc.predict_y(y1, y0, s)

y2, rhs, converged, iters  = alc.correct_y(
    y1,
    y0,
    b_ext,
    f_nl,
    df_nl_dx,
    df_nl_d_xdot,
    NH,
    n,
    N,
    s,
    M,
    C,
    K,
    alc_tol,
    alc_max_iter,
)

0.020798398058625038
0.007778256438749253
0.007499311778228723
0.00849228442419228
0.0011956226835935161
0.022813246682826863
0.016262755853139885
0.007360360660189084
0.0017803654326276677
0.008575156563237553
0.0033627844308078024
0.038225444041065156
0.030668753526794705
0.01939674516427611
0.014904363043260854
0.006448635105930267
0.003380605044365398
0.0036073552875600184
0.009849034936038083
0.006021335309765907


In [ ]:
omega2, z2 = y2[-1], y2[:-1]
omega2

np.complex128(2.2493506461365476+0j)

In [ ]:
np.linalg.norm(rhs), np.linalg.norm(y2), iters

(np.float64(0.04730969646127374), np.float64(2.183491621257844), 10)

# Check shapes

In [ ]:
z1, z0 = b_ext.toarray() + 0.01, b_ext.toarray()
omega1, omega0 = 2.01, 2.0
y1, y0 = np.concat((z1, [omega1])), np.concat((z0, [omega0]))
s = 0.1

(
    alc.get_P(y1, y0, s),
    alc.get_dP_dz(z1, z0),
    alc.get_dP_d_omega(omega1, omega0),
)

(np.float64(-0.008900000000000005),
 array([0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j,
        0.02+0.j, 0.02+0.j, 0.02+0.j, 0.02+0.j]),
 0.019999999999999574)

In [ ]:
db_dz = solve.get_db_nl_dz(omega0, z1, df_nl_dx, df_nl_d_xdot, NH, n, N)
db_dz.shape

(10, 10)

In [ ]:
A = freq.get_A(omega0, NH, M, C, K)
(
    solve.get_R(z1, omega1, A, f_nl, b_ext, NH, n, N).shape,
    solve.get_dR_dz(A, db_dz).shape,
    continuation.get_dR_d_omega(z1, omega1, df_nl_d_xdot, NH, n, N, M, C).shape,
)

((10,), (10, 10), (10,))

In [ ]:
omega = 1.1
A = freq.get_A(omega, NH, M, C, K)
z0 = solve.get_initial_guess(A, b_ext)

z, *_ = solve.solve_nonlinear(
    omega, z0, A, b_ext, f_nl, df_nl_dx, df_nl_d_xdot, NH, n, N
)
z

array([ 6.11575978e-18-9.19331064e-20j,  1.22315196e-18-1.83866213e-20j,
        9.09755566e-04-5.61088480e-01j, -2.80145909e-01-1.20532805e-01j,
        1.73748978e-18-6.57986679e-19j, -3.33977645e-20-2.45951689e-19j,
       -7.91917579e-04+1.46925578e-03j,  1.12907933e-04+7.79837428e-05j,
       -1.44206196e-18+5.33647630e-19j,  2.55959099e-20+8.79974466e-20j])